<a href="https://colab.research.google.com/github/Gab2697/Surface-classification-EMG/blob/main/EMG_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import
import pandas as pd
import tensorflow as tf
import numpy as np
import pickle
import matplotlib.pyplot as plt
import torch

from keras import layers 
from keras import models
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import regularizers

from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, Dropout
from tensorflow.keras.losses import binary_crossentropy, categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop


from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def open_pickle(path_pickle):
    f = open(path_pickle, 'rb')
    T = pickle.load(f)
    f.close()
    return T

def subject_wise_split(Participant,subject_wise,split=0.10,seed=42):
    np.random.seed(seed)
    if subject_wise:
        UniqParti=np.unique(Participant)
        num=np.round(UniqParti.shape[0]*split).astype('int64')
        np.random.shuffle(UniqParti)
        extract=UniqParti[0:num]
        test_index=np.array([],dtype='int64')
        for j in extract:
            test_index=np.append(test_index,np.where(Participant==j)[0])
        train_index=np.delete(np.arange(len(Participant)),test_index)
        np.random.shuffle(test_index)
        np.random.shuffle(train_index)

    else:
        I=np.arange(len(Participant)).astype('int64')
        np.random.shuffle(I)
        num=np.round(Participant.shape[0]*split).astype('int64')
        test_index=I[0:num]
        train_index=I[num:]
        extract=np.unique(Participant[test_index])
    return train_index,test_index,extract

def one_hot(y):
    uniq=np.unique(y)
    y_hot=np.zeros([y.shape[0],uniq.shape[0]])
    for i in range(len(uniq)):
        index=np.where(y==uniq[i])[0]
        y_hot[index,i]=1
    surface_name=uniq
    return y_hot

def CNN_test(input_shape,output_shape):
    model = tf.keras.models.Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def ANN(hid_layers,model,output,input_shape,activation_hid='relu'):
  ann = tf.keras.models.Sequential()
  ann.add(tf.keras.Input(shape=input_shape))
  for l in hid_layers:
    ann.add(tf.keras.layers.Dense(units=l,activation=activation_hid))
    if model=='classification':
      ann.add(tf.keras.layers.Dense(units=output,activation='softmax'))
      ann.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
    return ann

In [ ]:
#load pickle file
Ndata=open_pickle('/content/drive/MyDrive/Gab/Variables_saved/All_emg/data_good.pkl') 

In [ ]:
#Reshape into 3D arrays 
import math
data=np.zeros([Ndata.shape[0],7704,4])
Parameters=Ndata.keys()
for i in range(len(Parameters)-2):
  for j in range(len(Ndata[Parameters[i]])):
    data[j,:,i]=math.log(Ndata[Parameters[i]].to_numpy()[j].reshape(7704))

print(data.shape)

In [ ]:
for i in range(4):
  x=math.log(data[i])

In [ ]:
Participant= Ndata['Subject']
Surface= Ndata['Conditions']

In [ ]:
#seperate train and test set
train_index,test_index,extract=subject_wise_split(np.array(Participant),split=0.15,subject_wise=False,seed=5)
train_index,test_index=train_index.astype('int64'),test_index.astype('int64')   

X_train=data[train_index]
X_test=data[test_index]
y_train=Surface[train_index]
y_test=Surface[test_index]

In [ ]:
y_train=y_train.to_numpy()
y_test=y_test.to_numpy()

In [ ]:
#one hot encoding for label
X_train=np.asarray(X_train).astype(np.float32) 
y_train=one_hot(y_train)
X_test=np.asarray(X_test).astype(np.float32) 
y_test=one_hot(y_test)

In [ ]:
model=CNN_test(X_train.shape[1:],2)

In [ ]:
history= model.fit(X_train,y_train, epochs=10, batch_size=200,validation_data=(X_test,y_test))

Epoch 1/10
23/23 [==============================] - 147s 6s/step - loss: 8096.3428 - accuracy: 0.5038 - val_loss: 0.6933 - val_accuracy: 0.4834
Epoch 2/10
23/23 [==============================] - 143s 6s/step - loss: 0.6927 - accuracy: 0.4995 - val_loss: 0.6934 - val_accuracy: 0.4834
Epoch 3/10
23/23 [==============================] - 143s 6s/step - loss: 0.6927 - accuracy: 0.4995 - val_loss: 0.6933 - val_accuracy: 0.4834
Epoch 4/10
23/23 [==============================] - 138s 6s/step - loss: 0.6927 - accuracy: 0.4995 - val_loss: 0.6933 - val_accuracy: 0.4834
Epoch 5/10
14/23 [=================>............] - ETA: 53s - loss: 0.6924 - accuracy: 0.5018

KeyboardInterrupt: ignored

In [ ]:
from scipy import stats
Ndara_grass=Ndata[Ndata['Conditions'] == 'grass ']
Ndara_asphalt=Ndata[Ndata['Conditions'] == 'asphalt ']
rvs1 = stats.norm.rvs(loc=5, scale=10, size=500, random_state=Ndara_grass)
rvs2 = stats.norm.rvs(loc=5, scale=10, size=500, random_state=Ndara_asphalt)
stats.ttest_ind(rvs1, rvs2)

other

In [ ]:
#Condition and surface number instead of array
for i in range(Ndata['Conditions'].shape[0]):
    Ndata['Conditions'][i]=Ndata['Conditions'][i][0]
    Ndata['Subject'][i]=Ndata['Subject'][i][0]

In [ ]:
Surface=Ndata['Conditions']
Participant=Ndata['Subject']
del Ndata['Conditions']
del Ndata['Subject']

In [ ]:
#create dictionary with numpy array
data={}

for i in Ndata.keys():
  data[i]=np.zeros([5256,7704])
  for j in range(Ndata[i].shape[0]):
    data[i][j,:]=Ndata[i][j].transpose()[:,0]

In [ ]:
newdata={}
for i in data.keys():
    for j in range(8):
        newdata[f'{i}_{j}']=data[i][:,j*963:963+(j*963)]

In [ ]:
train_index,test_index,extract=subject_wise_split(np.array(Participant),split=0.15,subject_wise=False,seed=5)
train_index,test_index=train_index.astype('int64'),test_index.astype('int64')        

In [ ]:
cnnData={'X_train':{},'X_test':{},'y_train':{},'y_test':{}}

for i in newdata.keys():
  Data['X_train'][f'{i}']=newdata[train_index]
  Data['X_test'][f'{i}']=newdata[test_index]
  Data['y_train'][f'{i}']=Surface[train_index]
  Data['y_test'][f'{i}']=Surface[test_index]

In [ ]:
for i in Data['X_train'].keys():
    X_train.append(Data['X_train'][i])
    X_test.append(Data['X_test'][i])

In [ ]:
X_train=np.array(X_train)
X_test=np.array(X_test)

In [ ]:
x_train=X_train.reshape(X_train.shape[1],X_train.shape[0],1)
x_test=X_test.reshape(X_test.shape[1],X_test.shape[0],1)

In [ ]:
model=make_model_1D(x_train.shape[0],32,64,64,3,2,SGD(lr=0.01))
history= model.fit(x_train,y_train, epochs=10, batch_size=100,validation_data=(x_valid,y_valid))

Other stuff I tested

In [ ]:
#removing gravel for now
Ndata_no_gravel=Ndata[Ndata['Conditions'] != 'gravel ']
print('shape of data with the gravel surface:',Ndata.shape)
print('shape of data without the gravel surface:',pd.DataFrame(Ndata_no_gravel).shape)
np.unique(Ndata_no_gravel['Conditions'])

shape of data with the gravel surface: (5256, 6)
shape of data without the gravel surface: (5216, 6)


array(['asphalt', 'grass  '], dtype=object)

In [ ]:
# scikit-learn k-fold cross-validation
from numpy import array

# data sample
data = Ndata_no_gravel
# split into train test sets
n = len(pd.unique(Ndata_no_gravel['Subject'])) # number of participants
n_train= round(0.8*n) # number of participants for trianing
print('number of participants for the training set:', n_train)
n_test= n-n_train #number of participant for testing
#index of the last participant for trianing
n=Ndata_no_gravel.loc[Ndata_no_gravel['Subject'] == 'P28'].index
Ndata_train= Ndata_no_gravel.iloc[:n[-1],:] #select training set
Ndata_test= Ndata_no_gravel.iloc[n[-1]:,:] #select testing set
print('test set shape:', Ndata_test.shape)

#validation set
n_train2=round(n_train*0.8)
print('number of participants for the final training set:', n_train2)
n2=Ndata_train.loc[Ndata_train['Subject'] == 'P22'].index
Ndata_train2= Ndata_train.iloc[:n2[-1],:] #select final training set
Ndata_valid= Ndata_train.iloc[n2[-1]:,:] #select validation testing set
print('valid set shape:', Ndata_valid.shape, 'training set shape:', Ndata_train2.shape)

number of participants for the training set: 28
test set shape: (1859, 6)
number of participants for the final training set: 22
valid set shape: (965, 6) training set shape: (2392, 6)


In [ ]:
#seperate x data and it's label

Surface_train=Ndata_train2['Conditions']
Participant_train=Ndata_train2['Subject']
del Ndata_train2['Conditions']
del Ndata_train2['Subject']

Surface_valid=Ndata_valid['Conditions']
Participant_valid=Ndata_valid['Subject']
del Ndata_valid['Conditions']
del Ndata_valid['Subject']

# TA=Ndata['Trignosensor1TAEMG1Volts']
# Gastroc=Ndata['Trignosensor2GastrocEMG2Volts']
# RF=Ndata['Trignosensor3RFEMG3Volts']
# BF=Ndata['Trignosensor4BFEMG4Volts']

In [ ]:
x_train=Ndata_train2
y_train=Surface_train
x_valid=Ndata_valid
y_valid=Surface_valid